In [3]:
import os
import csv
from google.colab import drive
import pandas as pd
import numpy as np
import datetime as dt
from glob import glob
import argparse
from pathlib import Path
import json
from tqdm import tqdm, trange

In [4]:
def parse_args():
  parser = argparse.ArgumentParser(
      description='combine all raw data in to tie form data'
  )
  parser.add_argument(
      '--tie_mode',
      type=str,
      help='tie range, departclass means only count in class tie',
      default='departclass',
  )
  parser.add_argument(
      '--time_range',
      type=list,
      help='research time range',
      default=[(2012, 9, 30), (2015, 6, 30)],
  )
  parser.add_argument(
      '--from_raw',
      type=bool,
      help='generate all tie df from raw, if false from previous file',
      default=True,
  )
  parser.add_argument(
      '--ch_dir',
      type=str,
      default='/content/drive/MyDrive/fb_college/traid'
  )
  parser.add_argument(
      '--raw_dir',
      type=str,
      default='../raw/csv'
  )
  parser.add_argument(
      '--dtype_file',
      type=str,
      default='./tmp/dtype.json'
  )
  parser.add_argument(
      '--parse_dates_file',
      type=str,
      default='./tmp/parse_dates.json'
  )
  parser.add_argument(
      '--tie_correspond_file',
      type=str,
      default='./tmp/tie_correspond.json'
  )
  # --------------------------> raw data file
  parser.add_argument(
      '--survey_file',
      type=str,
      default='../raw/csv/survey.csv'
  )
  # --------------------------> preprocess file
  parser.add_argument(
      '--output_file',
      type=str,
      help='all tie file',
      default='./tmp/all_tie.csv'
  )
  args, unknown = parser.parse_known_args()
  return args
args = parse_args()

In [5]:
# 切換到工作目錄
os.chdir(args.ch_dir)
from utils import traid

In [6]:
# read tie columns dtype
dtype = json.loads(Path(args.dtype_file).read_text())
# read tie
tie_correspond = json.loads(Path(args.tie_correspond_file).read_text())
# 列出來要處理的檔案
tie_file_list = list(tie_correspond.keys())

In [7]:
# 如果預設要從 raw data 開始生成檔案
if args.from_raw:

  # 切換到儲存原始資料的目錄
  os.chdir(args.raw_dir)
  # 包含寫檔，執行時間5m7s
  # 遍歷每一個檔案
  for i in range(len(tie_file_list)):
    os.listdir()

    # 判斷時間欄位名稱
    # 檢查tie、merge哪個檔案包含時間的欄位，以及是哪種時間欄位
    if 'created_time' in tie_correspond[tie_file_list[i]]['tie_col']:
      # 若有created_time則讀檔時處理created_time為時間格式
      tie_parse_dates = ['created_time']
      m_parse_dates = False
      # 若有'created_time_date'則讀檔時處理其為時間格式 
    elif 'created_time_date' in tie_correspond[tie_file_list[i]]['tie_col']:
      tie_parse_dates = ['created_time_date'] # created_time_date
      m_parse_dates = False
    else:
      tie_parse_dates = False # 都沒有tie_df就不處理時間，代表時間欄位在要merge的檔案裡面
      # check merge file time columns is 'created_time' or 'created_time_date'
      if 'created_time' in tie_correspond[tie_file_list[i]]['merge_col']:
        m_parse_dates = ['created_time']
      elif 'created_time_date' in tie_correspond[tie_file_list[i]]['merge_col']:
        m_parse_dates = ['created_time_date']

    # read file
    tie_df = pd.read_csv(tie_file_list[i], # 讀取的檔案名稱
              usecols=tie_correspond[tie_file_list[i]]['tie_col'], # 該檔案用的欄位
              dtype=dtype, #資料型態設置
              parse_dates=tie_parse_dates) # 處理時間的欄位，先用頭一萬筆資料跑
    print(f"before merge {tie_file_list[i]} shape = ", tie_df.shape)
    # print(tie_file_list[i], tie_dic[tie_file_list[i]]['tie_col'])
    merge_df = pd.read_csv(tie_correspond[tie_file_list[i]]['m_file'],
              usecols=tie_correspond[tie_file_list[i]]['merge_col'],
              dtype=dtype,
              parse_dates=m_parse_dates)
    # 決定merge_on_column
    if (('-message_tags' in tie_file_list[i]) or
        ('_message_tags' in tie_file_list[i])): # 只有在評論當中標記人的檔案
      merge_on = ['OwnerFbid', 'Fbid', 'comment_id'] # 需要comment_id
    else:
      merge_on = ['OwnerFbid', 'Fbid'] # 其他都是Fbid而已
    
    # 處理重複資料，先按日期排列
    # 以first模式處理，留下最早的，刪掉其他重複的列
    if tie_parse_dates != False:
      tie_df = tie_df.sort_values(by=tie_parse_dates) # 如果有日期就按照日期排列
    if m_parse_dates != False:
      merge_df = merge_df.sort_values(by=m_parse_dates) # 如果有日期就按照日期排列

    # 清理tie_df, merge_file重複資料
    traid.duplicates_drop_report(tie_df, subset=None)
    merge_df = traid.duplicates_drop_report(merge_df, subset=merge_on)

    # 合併資料集
    tie_df = tie_df.merge(merge_df, how='left', on=merge_on)
    merge_on.remove('OwnerFbid') # 除了OwnerFbid其他的merge_on的欄位都不需要
    # print('merge_on', merge_on)
    tie_df = tie_df.drop(merge_on, axis=1) # 替除掉不需要的欄位
    # print('after drop', tie_df.columns)

    # 清理重複的資料
    tie_df = traid.duplicates_drop_report(tie_df, subset=None)

    # 重新命名欄位
    # print(tie_df.head())
    rename_dic = {'created_time': 'createdtime_date',
                  'created_time_date': 'createdtime_date',
                  'from_id_x': 'from_id',
                  'comments_from_id': 'from_id',
                  'id': 'from_id_p',
                  'tags_id': 'from_id_p',
                  'from_id_y': 'from_id_p',
                  'from_id_x': 'from_id'}
    # 評論是特例，from_id是被動，comments_from_id是主動
    if tie_file_list[i] == 'comments.csv':
      rename_dic = {'created_time': 'createdtime_date',
                    'created_time_date': 'createdtime_date',
                    'comments_from_id': 'from_id',
                    'from_id': 'from_id_p'}
    tie_df.rename(columns=rename_dic, inplace=True)

    # 新增欄位，代表這個是什麼tie
    tie_df['tie_type'] = i+1
    # split file name, drop '.csv'
    tie_df['tie_file'] = tie_file_list[i].split('.')[0]
    # tag or comment, split file name, drop '.csv', drop str before '_'
    # drop str before '-'
    tie_df['tag_or_com'] = tie_file_list[i].split('.')[0].split('_')[-1].split('-')[-1]
    # print(tie_file_list[i])
    # print(tie_df.columns)
    # # 缺失值處理
    print('before drop na:')
    print(tie_df.isnull().sum())
    tie_df = tie_df.dropna(axis=0)
    print('after drop na:')
    print(tie_df.isnull().sum())
    
    # 合併形成all_tie
    if i == 0:
      # 去除掉 network 當中的 loop
      tie_df = tie_df[tie_df['from_id'] != tie_df['from_id_p']]
      all_tie_df = tie_df
      print(f"all tie shape = ", all_tie_df.shape)
    else:
      # 去除掉 network 當中的 loop
      tie_df = tie_df[tie_df['from_id'] != tie_df['from_id_p']]
      all_tie_df = all_tie_df.append(tie_df)
      print(f"after merge {tie_file_list[i]} shape = ", tie_df.shape)
      print(f"all tie shape = ", all_tie_df.shape)

  # dealing with UTC time, convert it into datetime format
  all_tie_df = traid.UTC_to_datetime(all_tie_df, 'createdtime_date')

before merge message_tags.csv shape =  (89292, 3)
before drop na:
from_id_p           0
OwnerFbid           0
from_id             0
createdtime_date    0
tie_type            0
tie_file            0
tag_or_com          0
dtype: int64
after drop na:
from_id_p           0
OwnerFbid           0
from_id             0
createdtime_date    0
tie_type            0
tie_file            0
tag_or_com          0
dtype: int64
all tie shape =  (88543, 7)
before merge with_tags.csv shape =  (1365941, 3)
before drop na:
from_id_p           0
OwnerFbid           0
from_id             0
createdtime_date    0
tie_type            0
tie_file            0
tag_or_com          0
dtype: int64
after drop na:
from_id_p           0
OwnerFbid           0
from_id             0
createdtime_date    0
tie_type            0
tie_file            0
tag_or_com          0
dtype: int64
after merge with_tags.csv shape =  (1319059, 7)
all tie shape =  (1407602, 7)
before merge comments.csv shape =  (5177177, 4)
before drop na:
O

In [8]:
# 如果是從原始檔案做起，寫檔
if args.from_raw:
  # 切回工作目錄
  os.chdir(args.ch_dir)
  print('all_tie shape before drop loop', all_tie_df.shape)
  before_drop_nrow = all_tie_df.shape[0]
  all_tie_df = traid.drop_loop(all_tie_df, 'from_id', 'from_id_p')
  print('all_tie shape after drop loop', all_tie_df.shape)
  before_write_nrow = all_tie_df.shape[0]
  all_tie_df.to_csv(args.output_file, index=False)
else: # 否則讀取先前已經做好的檔案
  # all_tie_df = pd.read_csv(args.output_file, dtype=dtype)
  all_tie_df = pd.read_csv(args.output_file, dtype=dtype)

all_tie shape before drop loop (10109440, 7)
all_tie shape after drop loop (10109440, 7)


In [9]:
# 如果要單純討論單純討論inclass的的tie
if (args.tie_mode == 'departclass'):
  add_var = args.tie_mode
  add_var_id = 'OwnerFbid'
  tie_col = ['from_id', 'from_id_p']
  usecols = [add_var_id, add_var]
  survey_df = pd.read_csv(args.survey_file , dtype=dtype, usecols=usecols)

  # return df which have add from_id's departmentclass and from_id_p's departmentclass
  depart_df, add_var_list = traid.addVarToTie(all_tie_df, survey_df, tie_col, add_var_id , add_var)
  # keep if from and tie are from same departmentclass
  indepart_df = depart_df[depart_df[add_var_list[0]] == depart_df[add_var_list[1]]]
  indepart_df.to_csv('./tmp/all_tie_departclass.csv', index=False)

remove OwnerFbid in original_df
add_val_name:  from_id_departclass
add_val_name:  from_id_p_departclass


In [27]:
indepart_df

,from_id_p,from_id,createdtime_date,tie_type,tie_file,tag_or_com,from_id_departclass,from_id_p_departclass
0,1054096614651586,928607703886928,2012-11-06 21:45:16,2.0,with_tags,tags,801.0,801
1,1054096614651586,928607703886928,2012-11-06 21:45:31,2.0,with_tags,tags,801.0,801
2,1054096614651586,928607703886928,2012-11-06 21:45:16,2.0,with_tags,tags,801.0,801
3,1054096614651586,928607703886928,2012-11-06 21:45:31,2.0,with_tags,tags,801.0,801
4,1054096614651586,928607703886928,2012-11-06 21:39:47,2.0,with_tags,tags,801.0,801
...,...,...,...,...,...,...,...,...
723495,1001684923224152,1047176862000228,2013-06-27 00:16:52,8.0,uploaded_photos_tags,tags,5501.0,5501
723496,1001684923224152,1047176862000228,2013-10-24 22:32:39,8.0,uploaded_photos_tags,tags,5501.0,5501
723497,1001684923224152,1047176862000228,2013-12-25 22:41:17,8.0,uploaded_photos_tags,tags,5501.0,5501
723498,1001684923224152,1047176862000228,2014-05-14 02:14:06,8.0,uploaded_photos_tags,tags,5501.0,5501
